# Plots for the paper concerning the EWino scan

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,shutil
from pandas.io.json import json_normalize
import pyslha
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from collections import OrderedDict
import seaborn as sns
from scipy.interpolate import interp1d
from IPython.display import display, Markdown

pd.option_context('display.max_columns', -1)

pd.options.mode.chained_assignment = None #Disable copy warnings
# plt.style.use('fivethirtyeight') #Set style
# mpl.rcParams.update({'figure.figsize' : (15,10)})  #Set general plotting options
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

#### Create txname/process translator

In [2]:
processDict = {'TChiWZ' : r'$\tilde{\chi}_1^\pm \, \tilde{\chi}_2^0 \to W^{(*)} Z^{(*)}\, \tilde{\chi}_1^0 \tilde{\chi}_1^0$',
              'TChiWH' : r'$\tilde{\chi}_1^\pm \, \tilde{\chi}_2^0 \to W h\, \tilde{\chi}_1^0 \tilde{\chi}_1^0$',
              'TChiWW' : r'$\tilde{\chi}_1^\pm \, \tilde{\chi}_1^\mp \to W W \, \tilde{\chi}_1^0 \tilde{\chi}_1^0$',
              'TDTM1F,TDTM2F' : r'$\tilde{\chi}_1^\pm \to \pi^\pm \tilde{\chi}_1^0$, DT'}
def convertLabel(label):
    newlabel = label[:]
    #Replace txnames by process
    for txlabel,proc in processDict.items():
        newlabel = newlabel.replace(txlabel,proc) 
    #Remove off-shell Ws and Zs from ATLAS analyses (since they only appear in on-shell cases)
    if 'ATLAS' in newlabel:
        newlabel = newlabel.replace('W^{(*)} Z^{(*)}', 'W Z')
    return newlabel

In [ ]:
#Merge with SModelS DataFrame
dataDF = pd.read_pickle('EWino_scanRandom.pcl')
# print(dataDF.columns.values.tolist()) #Print all columns names

#### Set r = 0 for points with no results 

In [ ]:
#Set points without results with r == 0
dataDF.fillna(value={'ExptRes.result0.r' : 0.0},inplace=True)
#Sort points according to r value:
dataDF = dataDF.sort_values(by=['ExptRes.result0.r'],ascending=False)

#### Define exclusions

In [ ]:
#Add simple flag for excluded points:
dataDF['excludedLHC'] = dataDF['ExptRes.result0.r'] > 1.0
dataDF['excludedRelic'] = dataDF['Omega'] > 0.13
#Xenon 1T curve (from micromegas):
data_1t_lnM = [1.790E+00, 1.885E+00, 2.006E+00, 2.148E+00, 2.297E+00, 2.449E+00, 2.598E+00, 2.783E+00, 2.961E+00, 3.117E+00, 3.302E+00, 3.444E+00, 3.614E+00, 3.796E+00, 4.037E+00, 4.268E+00, 4.581E+00, 4.904E+00, 5.178E+00, 5.441E+00, 5.768E+00, 6.095E+00, 6.369E+00, 6.550E+00, 6.891E+00]
data_1t_lnS = [1.040E+00, 2.794E-02,-1.003E+00,-2.035E+00,-2.888E+00,-3.591E+00,-4.144E+00,-4.717E+00,-5.111E+00,-5.326E+00,-5.458E+00,-5.486E+00,-5.440E+00,-5.375E+00,-5.178E+00,-4.982E+00,-4.711E+00,-4.412E+00,-4.150E+00,-3.925E+00,-3.598E+00,-3.270E+00,-3.009E+00,-2.812E+00,-2.485E+00]
xenonF = interp1d(data_1t_lnM,data_1t_lnS,
                  bounds_error=False,fill_value="extrapolate") #interpolate in log
@np.vectorize
def xenonUL(mDM):
    return 1e-8*np.exp(xenonF(np.log(mDM))) #90% upper limit on sigmaNucleon (pb)

rDD = (dataDF['Omega']/0.12)*(dataDF['proton_SI']+dataDF['neutron_SI'])/2.0
rDD = rDD/xenonUL(dataDF['mass.1000022'])
dataDF['excludedDD'] = (rDD > 1)

#### Add c*tau column for charginos

In [ ]:
dataDF['ctau.1000024'] = 1.967e-16/dataDF['width.1000024']

### Get points excluded:

In [ ]:
excluded = dataDF[dataDF['excludedLHC'] == True]
# excludedDM = dataDF[(dataDF['excludedLHC'] == False) & (dataDF['excludedRelic'] == True)]
# excludedDD = dataDF[(dataDF['excludedLHC'] == False) & (dataDF['excludedRelic'] == False) 
#                     & (dataDF['excludedDD'] == True)]

excludedDM = dataDF[(dataDF['excludedRelic'] == True)]
excludedDD = dataDF[(dataDF['excludedDD'] == True)]


allowed = dataDF[(dataDF['excludedLHC'] == False) & (dataDF['excludedRelic'] == False) 
                 & (dataDF['excludedDD'] == False)]

allowedLHC = dataDF[(dataDF['excludedLHC'] == False)]


print('Total number of points = %i' %len(dataDF))
print('Total excluded (LHC) = %i'%(len(excluded)))
print('Total excluded (relic) = %i'%(len(excludedDM)))
print('Total excluded (DD) = %i'%(len(excludedDD)))
print('Total allowed = %i\n'%(len(allowed)))

# print('Total excluded (r > %1.2f) = %i'%(rscale,len(excludedSC)))
# print('Total allowed (r > %1.2f) = %i'%(rscale,len(allowedSC)))


### Create analysis label for excluded points

In [ ]:
anas = excluded['ExptRes.result0.AnalysisID'].value_counts()
anas.sort_values(inplace=True,ascending=False)

    
labels = []
for index, row in excluded.iterrows():
    ana = row['ExptRes.result0.AnalysisID']
    txList = sorted(row['ExptRes.result0.TxNames'])
    label = ana + ' (' + ','.join(txList) + ')'
    label = label.replace('off','') #Group together on and off-shell topologies
    labels.append(label)
excluded['label'] = labels


In [ ]:
nexcluded = [[len(excluded[excluded['label'] == label]),label] for label in excluded['label'].unique()]
nexcluded = sorted(nexcluded,reverse=True)
nexcluded = np.array(nexcluded)
for label in nexcluded[:,1]:
    print(label+': %i'%len(excluded[excluded['label'] == label]))

### Define colors and plotting order

In [ ]:
colors = sns.color_palette("Paired",len(excluded['label'].unique()))
colorsPink = sns.color_palette("colorblind",8)
colorDict = {'ATLAS-SUSY-2016-06 (TDTM1F,TDTM2F)' : colorsPink[4], #light pink
             'ATLAS-SUSY-2019-08 (TChiWH)' : colors[5], #red
             'CMS-SUS-17-004 (TChiWZ)' : colors[1], #dark blue
             'ATLAS-SUSY-2018-32 (TChiWW)' : colors[3], #dark green
             'ATLAS-SUSY-2018-06 (TChiWZ)' : colors[0], #light blue
             'CMS-EXO-19-010 (TDTM1F,TDTM2F)' : colorsPink[6], #dark pink
             'Other' : colors[9] #dark purple
             }
#Define plotting order:
plottingOrder = ['ATLAS-SUSY-2016-06 (TDTM1F,TDTM2F)','CMS-EXO-19-010 (TDTM1F,TDTM2F)',
                 'ATLAS-SUSY-2019-08 (TChiWH)', 'CMS-SUS-17-004 (TChiWZ)',
                 'ATLAS-SUSY-2018-06 (TChiWZ)','ATLAS-SUSY-2018-32 (TChiWW)']
#Add other analyses
for ana in excluded['label'].unique():
    if not ana in plottingOrder:
        plottingOrder.append(ana)

### Get relevant official exclusion curves

In [ ]:
excCurves = {}
excATLAS = np.genfromtxt('../data/HEPData-ins1641262-v4-Exclusion_contour_EW_2_obs_conv.txt',
                       names=True)

excCurves['ATLAS-SUSY-2016-06 (TDTM1F,TDTM2F)'] = np.array(list(zip(excATLAS['mC1_GeV'],1.967e-16/excATLAS['width_GeV'])))


excCurves['CMS-SUS-17-004 (TChiWZ)'] =  np.genfromtxt('../data/CMS_SUS_17_004_TChiWZ_obs.csv', 
                                                      delimiter=',')
excCurves['ATLAS-SUSY-2016-24 (TChiWZ)'] = np.genfromtxt('../data/HEPData-ins1658902-v1-Table_20.csv',
                       delimiter=',', skip_header=9)
excCurves['ATLAS-SUSY-2019-08 (TChiWH)'] = np.genfromtxt('../data/HEPData-ins1755298-v1-Observed_limit_1lbb.csv',
                       delimiter=',', skip_header=11)
excCurves['ATLAS-SUSY-2018-32 (TChiWW)'] = np.genfromtxt('../data/HEPData-ins1750597-v1-Exclusion_contour_(obs)_1.csv',
                       delimiter=',', skip_header=10)
excCurves['CMS-SUS-13-006 (TChiWZ)'] = np.genfromtxt('../data/CMS_SUS_13_006_TChiWZ_obs.csv', 
                                                      delimiter=',')

excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)'] = np.genfromtxt('../data/CMS_SUS_19_010_TChiWZ_obs.csv', 
                                                      delimiter=',')
#Fix column order:
excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)'][:,[0,1]] = excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)'][:,[1,0]]
#Convert width to lifetime
excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)'][:,1]= 1.967e-16/excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)'][:,1]


### Chargino lifetime

In [ ]:
cm = plt.cm.get_cmap('RdYlBu_r')
plt.figure(figsize=(10,6))
x = dataDF['extpar.2']/dataDF['extpar.23']
y = dataDF['extpar.2']/dataDF['extpar.1']
c = dataDF['ctau.1000024']
x = x[c.argsort()]
y = y[c.argsort()]
c = c[c.argsort()]

plt.scatter(x,y,c=c,s=20,cmap=cm, norm=LogNorm(vmin=1e-8,vmax=1e-1))
# plt.plot([x.min(),x.max()],[x.min(),x.max()],'--')
plt.colorbar(label=r'$c\tau$ (m)')
plt.xlabel(r'$M_2/\mu$ (GeV)')
plt.ylabel(r'$M_2/M_1$ (GeV)')
# plt.ylim(-1,10)
plt.xscale('log')
plt.yscale('log')
plt.text(0.7,10.0,r"{\bf Bino LSP}",fontsize=16,color='white')
plt.text(0.05,0.1,r"{\bf Wino LSP}",fontsize=16,color='white')
plt.text(2.,0.6,r"{\bf Higgsino LSP}",fontsize=16)

plt.tight_layout()
plt.savefig('charginoLifetime.png')
plt.show()

In [ ]:
print(dataDF['ctau.1000024'].max())

### Neutralino Branching ratio

In [ ]:
deleteCurves = ['CMS-EXO-19-010','ATLAS-SUSY-2016-06','ATLAS-SUSY-2016-24','CMS-SUS-13-006']
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)

plt.figure(figsize=(10,6))



cm = plt.cm.get_cmap('RdYlBu_r')
x = dataDF['mass.1000024']
y = dataDF['mass.1000022']
c = dataDF['BRs.N2.Z,N1']
for ana in anas.index:
    if not ana in excCurves: continue
    if any(val in ana for val in deleteCurves): continue
    curve = excCurves[ana]
    color = colorDict[ana]
    plt.plot(curve[:,0],curve[:,1],linewidth=3,linestyle='-',color=color)

plt.scatter(x,y,c=c,s=50,cmap=cm)
# plt.plot([x.min(),x.max()],[x.min(),x.max()],'--')
plt.colorbar(label=r'$BR(\tilde \chi_2^0 \to \tilde \chi_1^0 +Z)$')
plt.xlabel(r'$m_{\tilde{\chi}_1^\pm}$ (GeV)')
plt.ylabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.ylim(0,600)
plt.xlim(0,800)
plt.tight_layout()
plt.show()

In [ ]:
deleteCurves = ['CMS-EXO-19-010','ATLAS-SUSY-2016-06','ATLAS-SUSY-2016-24','CMS-SUS-13-006']
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)

plt.figure(figsize=(10,6))



cm = plt.cm.get_cmap('RdYlBu_r')
x = dataDF['mass.1000024']
y = dataDF['mass.1000022']
c = dataDF['BRs.N2.higgs,N1']
for ana in anas.index:
    if not ana in excCurves: continue
    if any(val in ana for val in deleteCurves): continue
    curve = excCurves[ana]
    color = colorDict[ana]
    plt.plot(curve[:,0],curve[:,1],linewidth=3,linestyle='-',color=color)

plt.scatter(x,y,c=c,s=50,cmap=cm)
# plt.plot([x.min(),x.max()],[x.min(),x.max()],'--')
plt.colorbar(label=r'$BR(\tilde \chi_2^0 \to \tilde \chi_1^0 +H)$')
plt.xlabel(r'$m_{\tilde{\chi}_1^\pm}$ (GeV)')
plt.ylabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.ylim(0,600)
plt.xlim(0,800)
plt.tight_layout()
plt.show()

### Plot excluded points grouped by most excluding analysis

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)

plt.figure(figsize=(10,6))

x = allowedLHC['mass.1000024']
y = allowedLHC['ctau.1000024']
c_other = 'lightgray'
plt.scatter(x,y,color=c_other,label='All',s=60)

otherLabel = 'Others'
for ana in plottingOrder:
    x = excluded[excluded['label'] == ana]['mass.1000024']
    y = excluded[excluded['label'] == ana]['ctau.1000024']
    if not ana in colorDict:
        color = colorDict['Other']        
        plt.scatter(x,y,color=color,label=otherLabel,s=60)
        otherLabel = None #suppress repeated label
    else:
        color = colorDict[ana]
        label = convertLabel(ana)
        plt.scatter(x,y,color=color,label=label,s=60)

    
    
    
#Add official DT exclusion curve
offExc = excCurves['ATLAS-SUSY-2016-06 (TDTM1F,TDTM2F)']
c = colorDict['ATLAS-SUSY-2016-06 (TDTM1F,TDTM2F)']
plt.plot(offExc[:,0],offExc[:,1],label='ATLAS-SUSY-2016-06 (DT)',linewidth=3,linestyle='-',color='black')
offExc = excCurves['CMS-EXO-19-010 (TDTM1F,TDTM2F)']
c = colorDict['CMS-EXO-19-010 (TDTM1F,TDTM2F)']
plt.plot(offExc[:,0],offExc[:,1],label='CMS-EXO-19-010 (DT)',linewidth=3,linestyle='--',color='black')

lgd = plt.legend(loc=(0.65,0.4), framealpha=1.,fontsize=15)
plt.yscale('log')
plt.ylim(1e-16,10)
# plt.ylim(1e-2,0.1)
plt.xlim(0,1000)
# plt.xticks([0,200.,300.,400.,450.,500.,550.])
# plt.grid()
plt.xlabel(r'$m_{\tilde{\chi}_1^\pm}$ (GeV)')
plt.ylabel(r'$c \tau_{\tilde{\chi}_1^\pm}$ (m)')
plt.yticks([1e-16,1e-14,1e-12,1e-10,1e-8,1e-6,1e-4,1e-2,1e0])
plt.tight_layout()
plt.savefig('EWino_exclusion_ctau.png')
plt.show()

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)

plt.figure(figsize=(10,6))

x = allowedLHC['mass.1000024']
y = allowedLHC['mass.1000022']
plt.scatter(x,y,color=c_other,label='All',s=60)


deleteCurves = ['CMS-EXO-19-010','ATLAS-SUSY-2016-06','ATLAS-SUSY-2016-24','CMS-SUS-13-006']

otherLabel = 'Others'
for ana in plottingOrder:
    x = excluded[excluded['label'] == ana]['mass.1000024']
    y = excluded[excluded['label'] == ana]['mass.1000022']
    if not ana in colorDict:
        color = colorDict['Other']        
        plt.scatter(x,y,color=color,label=otherLabel,s=60)
        otherLabel = None #suppress repeated label
    else:
        color = colorDict[ana]
        label = convertLabel(ana)
        plt.scatter(x,y,color=color,label=label,s=60)

for ana in anas.index:
    if not ana in excCurves: continue
    if any(val in ana for val in deleteCurves): continue
    curve = excCurves[ana]
    color = colorDict[ana]
    plt.plot(curve[:,0],curve[:,1],linewidth=3,linestyle='-',color=color)
        

lgd = plt.legend(loc=(0.65,0.5), framealpha=1.,fontsize=15)

plt.ylim(0,700)
plt.xlim(50,1000)
plt.xlabel(r'$m_{\tilde{\chi}_1^\pm}$ (GeV)')
plt.ylabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.tight_layout()
plt.savefig('EWino_exclusion_mass.png')
plt.show()

## Plots for Allowed points

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)
plt.figure(figsize=(10,6))

x = np.array(allowedLHC['mass.1000024'])
y = np.array(allowedLHC['mass.1000022'])
# z = allowedLHC['ExptRes.result0.r']
z = np.array(allowedLHC['ExptRes.result0.r']/allowedLHC['xsecRatio'])

x = x[z.argsort()]
y = y[z.argsort()]
z = z[z.argsort()]   

cm = plt.cm.get_cmap('RdYlBu')
cm = plt.cm.get_cmap('Greys')
cm = plt.cm.get_cmap('PiYG', 11) 
cm = plt.cm.get_cmap('PuOr', 11) 
plt.scatter(x,y,c=z,s=20,cmap=cm,vmin=0.0,vmax=2.0)

plt.plot([90,500],[0,410],linestyle='--',c='black')


for ana in anas.index:
    if 'TDT' in ana: continue #Do not show disappearing track curve
    if not ana in excCurves: continue
    if any(val in ana for val in deleteCurves): continue
    curve = excCurves[ana]
    color = colorDict[ana]
    label = convertLabel(ana)
    plt.plot(curve[:,0],curve[:,1],linewidth=5,linestyle='-',color=color,label=label)
        

lgd = plt.legend(loc=(0.05,0.75), framealpha=1.,fontsize=15)

plt.ylim(0,400)
plt.xlim(50,800)

plt.xlabel(r'$m_{\tilde{\chi}_1^\pm}$ (GeV)')
plt.ylabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.colorbar(label = r'$r_{\rm{max}} \times \sigma_{\tilde \chi \tilde \chi}^{\rm wino}/\sigma_{\tilde\chi \tilde\chi}$')
# plt.title("Points allowed by LHC")
plt.tight_layout()
plt.savefig('EWino_allowed_xsec.png')
plt.show()

### Dark Matter Plots

#### Relic Density

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)

plt.figure(figsize=(10,6))

xAll = allowedLHC['extpar.1']/allowedLHC[['extpar.2','extpar.23']].min(axis=1)
yAll = allowedLHC['Omega']
plt.scatter(xAll,yAll,color=c_other,label='All',s=60)


deleteCurves = ['CMS-EXO-19-010','ATLAS-SUSY-2016-06','ATLAS-SUSY-2016-24','CMS-SUS-13-006']

otherLabel = 'Others'
for ana in plottingOrder:
    excludedAna = excluded[excluded['label'] == ana]
    x = excludedAna['extpar.1']/excludedAna[['extpar.2','extpar.23']].min(axis=1)
    y = excludedAna['Omega']
    if not ana in colorDict:
        color = colorDict['Other']        
        plt.scatter(x,y,color=color,label=otherLabel,s=60)
        otherLabel = None #suppress repeated label
    else:
        color = colorDict[ana]
        label = convertLabel(ana)
        plt.scatter(x,y,color=color,label=label,s=60)

plt.hlines(y=0.13,xmin=xAll.min(),xmax=xAll.max(),linestyle='-',linewidth=2,color='black')        
plt.vlines(x=1.0,ymin=yAll.min(),ymax=yAll.max(),linestyle='--',linewidth=3,color='black',alpha=0.5)        

plt.text(5e-3,0.18,r"$\Omega h^2 = 0.13$",fontsize=18)
plt.text(1.1e-1,0.1e-2,r"{\bf Bino LSP}",fontsize=16)
plt.text(1.3,0.1e-2,r"{\bf Wino/Higgsino LSP}",fontsize=16)

lgd = plt.legend(loc=(0.65,0.5), framealpha=1.,fontsize=15)

plt.yscale('log')
plt.xscale('log')
# plt.xlim(0,4)
plt.xlabel(r'$M_1/{\rm min}(M_2,\mu)$')
plt.ylabel(r'$\Omega h^2$')
plt.tight_layout()
plt.savefig('EWino_exclusion_omega.png')
plt.show()

#### Direct Detection

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)
mDMlist = np.logspace(1,3.5,100)
sigUL = xenonUL(mDMlist)

plt.figure(figsize=(12,6))

c_other = 'lightgray'
c_dm = 'darkgray'
c_dd = 'gray'

x = allowed['mass.1000022']
y = (allowed['Omega']/0.12)*allowed['proton_SI']
plt.scatter(x,y,color=c_other,label='All',s=60)
x = excludedDD['mass.1000022']
y = (excludedDD['Omega']/0.12)*excludedDD['proton_SI']
plt.scatter(x,y,color=c_other,label=None,s=60)


        
x = excludedDM['mass.1000022']
y = (excludedDM['Omega']/0.12)*excludedDM['proton_SI']
plt.scatter(x,y,color=c_dm,label=r'Excluded ($\Omega h^2 > 0.13$)',s=60)




# x = excludedDD['mass.1000022']
# y = (excludedDD['Omega']/0.12)*excludedDD['proton_SI']
# plt.scatter(x,y,color=c_dd,label='Excluded (DD)',s=60)


otherLabel = 'Others'
for ana in plottingOrder:
    excludedAna = excluded[excluded['label'] == ana]
    excludedAna = excludedAna[excludedAna['excludedRelic'] == False] #Remove points excluded by Relic
    x = excludedAna['mass.1000022']
    y = (excludedAna['Omega']/0.12)*excludedAna['proton_SI']
    if len(excludedAna) == 0: continue
    if not ana in colorDict:
        color = colorDict['Other']        
        plt.scatter(x,y,color=color,label=otherLabel,s=60)
        otherLabel = None #suppress repeated label
    else:
        color = colorDict[ana]
        label = convertLabel(ana)
        plt.scatter(x,y,color=color,label=label,s=60)        


    
# x = allowed['mass.1000022']
# y = (allowed['Omega']/0.12)*allowed['proton_SI']
# plt.scatter(x,y,color=c_other,label='Allowed',s=60)

        
plt.plot(mDMlist,sigUL,c='black',label='Xenon 1T',linewidth=3)


lgd = plt.legend(loc=(0.85,0.45), framealpha=1.,fontsize=15)

# plt.ylim(0,1)
plt.xlim(10,1000)
plt.xlabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.ylabel(r'$(\Omega h^2/0.12) \sigma_{\rm{SI}}$ (pb)')
plt.yscale('log')
plt.xscale('log')
plt.tight_layout()
plt.savefig('EWino_exclusion_DD.png')
plt.show()

In [ ]:
anas = excluded['label'].value_counts()
anas.sort_values(inplace=True,ascending=False)
mDMlist = np.logspace(1,3.5,100)
sigUL = xenonUL(mDMlist)

plt.figure(figsize=(13,6))

c_other = 'lightgray'
c_dm = 'darkgray'
c_dd = 'gray'


x = excludedDM['mass.1000022']
y = (excludedDM['Omega']/0.12)*excludedDM['proton_SI']
plt.scatter(x,y,color=c_dm,label='Excluded (DM)',s=60)


x = excludedDD['mass.1000022']
y = (excludedDD['Omega']/0.12)*excludedDD['proton_SI']
plt.scatter(x,y,color=c_dm,label=None,s=60)


otherLabel = 'Others'
for ana in plottingOrder:
    x = excluded[excluded['label'] == ana]['mass.1000022']
    y = (excluded[excluded['label'] == ana]['Omega']/0.12)*excluded[excluded['label'] == ana]['proton_SI']
    if not ana in colorDict:
        color = colorDict['Other']        
        plt.scatter(x,y,color=color,label=otherLabel,s=60)
        otherLabel = None #suppress repeated label
    else:
        color = colorDict[ana]
        label = convertLabel(ana)
        plt.scatter(x,y,color=color,label=label,s=60)        

    
# x = allowed['mass.1000022']
# y = (allowed['Omega']/0.12)*allowed['proton_SI']
# plt.scatter(x,y,color=c_other,label='Allowed',s=60)

x = allowed['mass.1000022']
y = (allowed['Omega']/0.12)*allowed['proton_SI']
plt.scatter(x,y,color=c_other,label='Allowed',s=60)

        
plt.plot(mDMlist,sigUL,c='red',label='Xenon 1T',linewidth=3)


lgd = plt.legend(loc=(0.98,0.2), framealpha=1.,fontsize=15)

# plt.ylim(0,1)
plt.xlim(10,4000)
plt.xlabel(r'$m_{\tilde{\chi}_1^0}$ (GeV)')
plt.ylabel(r'$(\Omega h^2/0.12) \sigma_{SI}$ (pb)')
plt.yscale('log')
plt.xscale('log')
plt.tight_layout()
plt.show()

## Plots Concerning DM nature

### Large mu scenario

In [ ]:
largeMu = dataDF[(dataDF['extpar.23'] > 3*dataDF['extpar.2']) & (dataDF['extpar.23'] > 3*dataDF['extpar.1'])]

In [ ]:
fig = plt.figure(figsize=(10,6))


x = np.array(largeMu['extpar.1']/largeMu['extpar.2'])
y = np.array(largeMu['Omega'])    
z = np.array(largeMu['xsecRatio'])
x = x[z.argsort()]
y = y[z.argsort()]
z = z[z.argsort()]             

cm = plt.cm.get_cmap('RdYlBu')
a = plt.scatter(x,y,c=z,s=50,cmap=cm,vmin=0,vmax=1)
plt.hlines(y=0.12,xmin=min(x),xmax=max(x),linestyle='--')
plt.yscale('log')
plt.xlim(0,4)
plt.xlabel(r'$M_1/M_2$')
plt.ylabel(r'$\Omega h^2$')

plt.colorbar(label= r'$\sigma_{\rm{EW}}/\sigma_{\rm{EW}}^{\rm{wino}}$')
plt.text(2.0,1e4,r"$\mu > 3 \max(M_1,M_2)$")
plt.tight_layout()
plt.show()

### Light Chargino scenario

In [ ]:
lightC1 = dataDF[(dataDF['mass.1000023'] < 1000.0)]

In [ ]:
plt.figure(figsize=(10,6))

x = np.array(lightC1['extpar.1']/lightC1['extpar.2'])
y = np.array(lightC1['Omega'])
z = np.array(lightC1['xsecRatio'])

x = x[z.argsort()]
y = y[z.argsort()]
z = z[z.argsort()]     


cm = plt.cm.get_cmap('RdYlBu')
# cm = plt.cm.get_cmap('Greys')
plt.scatter(x,y,c=z,s=50,cmap=cm,vmin=0,vmax=1)
plt.colorbar(label= r'$\sigma_{\rm{EW}}/\sigma_{\rm{EW}}^{\rm{wino}}$')

xLHC = np.array(excluded['extpar.1']/excluded['extpar.2'])
yLHC = np.array(excluded['Omega'])
plt.scatter(xLHC,yLHC,c='black',s=50,marker='x',label='Excluded (LHC)')

plt.hlines(y=0.12,xmin=min(x),xmax=max(x),linestyle='--')
plt.yscale('log')
# plt.ylim(0,200)
plt.xlim(0,4)
plt.xlabel(r'$M_1/M_2$')
plt.ylabel(r'$\Omega h^2$')
plt.text(1.5,1000,r"$m_{\tilde{\chi}_1^\pm} < 1$ TeV",fontsize=20)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# # Count points with 0.7 <= r < 1
# border = dataDF[(dataDF['ExptRes.result0.r'] > 0.7) & (dataDF['ExptRes.result0.r'] < 1.0)]
# print('%i files found' %len(border))
# #Copy files to ../data/slha_scanRandom_border
# for f in border['filename']:
#     f_old = os.path.join('../data/slha_scanRandom/',f)
#     f_new = os.path.join('../data/slha_scanRandom_border/',f)
#     shutil.copyfile(f_old,f_new)